### IMPORTING LIBRARY

In [1]:
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer # It has the ability to lemmatize.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout

nltk.download('punkt')# required package for tokenization
nltk.download('wordnet')# word database
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\RITWIK
[nltk_data]     SRIVASTAVA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\RITWIK
[nltk_data]     SRIVASTAVA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\RITWIK
[nltk_data]     SRIVASTAVA\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Step two: Creating a JSON file

In [ ]:
data = {"intents": [

            {"tag": "age",
            "patterns": ["how old are you?"],
            "responses": ["I am 2 years old and my birthday was yesterday"]
            },
            {"tag": "greeting",
            "patterns": [ "Hi", "Hello", "Hey"],
            "responses": ["Hi there", "Hello", "Hi :)"],
            },
            {"tag": "goodbye",
            "patterns": [ "bye", "later"],
            "responses": ["Bye", "take care"]
            },
            {"tag": "name",
            "patterns": ["what's your name?", "who are you?"],
            "responses": ["I have no name yet," "You can give me one, and I will appreciate it"]
            }

]}

### Step three: Processing data

In [ ]:
lm = WordNetLemmatizer() #for getting words
#list
ourclass = []
newwords = []
documentX = []
documentY = []

# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.

for intent in data['intents'] :
    for pattern in intent['patterns'] :
        ournewtkns = nltk.word_tokenize(pattern)# tokenize the patterns
        newwords.extend(ournewtkns)#extends our tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])
        
    if intent['tag'] not in ourclass : #add unexisting tag
        ourclass.append(intent['tag'])
        

newwords = [lm.lemmatize(word.lower()) for word in newwords if word not in string.punctuation]
newwords = sorted(set(newwords)) #sorting words
ourclass = sorted(set(ourclass))

### Step four: Designing a neural network model

The code below is used to turn our data into numerical values using bag of words (BoW) encoding system:

In [ ]:
trainingData = [] #trianing list array
outempty = [0] * len(ourclass)
#  bow model
for idx , doc in enumerate(documentX) :
    bagofwords = []
    text = lm.lemmatize(doc.lower())
    for word in newwords :
        bagofwords.append(1) if word in text else bagofwords.append(0)
        
        
    outputrow = list(outempty)
    outputrow[ourclass.index(documentY[idx])] = 1
    trainingData.append([bagofwords , outputrow])

random.shuffle(trainingData)
trainingData = np.array(trainingData , dtype = object)

x = np.array(list(trainingData[:,0])) # first training phase
y = np.array(list(trainingData[:,1])) # second training phase  

In [ ]:
ishape = (len(x[0]),)
oshape = (len(y[0]))

#parameter definantion

model = Sequential()

model.add(Dense(128 , input_shape = ishape, activation = "relu"))

model.add(Dropout(0.5)) # will drop 50% of the neurons to solve overfitting

model.add(Dense(64,activation = "relu"))
model.add(Dropout(0.3))
model.add(Dense(oshape,activation = "softmax"))

md = tf.keras.optimizers.Adam(learning_rate=0.01 , decay = 1e-6)

model.compile(loss = "categorical_crossentropy",
                optimizer = md ,
                metrics = ['accuracy'])

print(model.summary())

model.fit(x,y,epochs=200,verbose = 1)


### Step five: Building useful features

In [ ]:
def ourtext(text) : 
    newtkns = nltk.word_tokenize(text)
    newtkns = [lm.lemmatize(word) for word in newtkns]
    return newtkns

def wordbag (text,vocab) :
    newtkns = ourtext(text)
    bagofwords = [0] * len(vocab)
    for w in newtkns :
        for idx , doc in enumerate(vocab) :
            if word == w :
                bagofwords[idx] = 1
    return np.array(bagofwords)

def Pclass (text,vocab,labels) :
    bagofwords = wordbag(text,vocab)
    ourresult = model.predict(np.array([bagofwords]))[0]
    newthresh = 0.2
    yp = [[idx,res] for idx , res in enumerate(ourresult)if res > newthresh]
    
    yp.sort(key = lambda x: x[1],reverse = True)
    newlist = []
    for r in yp :
        newlist.append(labels[r[0]])
    return newlist


def getres(firstlist , fjson) :
    tag =firstlist[0]           
    listofintents = fjson['intents']
    for i in listofintents :
        if i["tag"] == tag :
            ourresult = random.choice(i["responses"])
            break
    return ourresult    

In [8]:
while True:
    newMessage = input("")
    intents = Pclass(newMessage, newwords, ourclass)
    ourResult = getres(intents, data)
    print(ourResult)